[Reference](https://medium.com/@ethan.duong1120/stock-data-analysis-project-python-1bf2c51b615f)

In [1]:
!pip instalpl yfinance
import pandas as pd
import yfinance as yf
import plotly.express as px
from datetime import datetime

ERROR: unknown command "instalpl" - maybe you meant "install"


In [2]:
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

In [3]:
tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL']

In [4]:
df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)
df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])

df = df.reset_index() # we have to reset index

print(df.head(100))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
   Ticker       Date        Open        High         Low       Close  \
0    AAPL 2023-05-17  171.710007  172.929993  170.419998  172.690002   
1    AAPL 2023-05-18  173.000000  175.240005  172.580002  175.050003   
2    AAPL 2023-05-19  176.389999  176.389999  174.940002  175.160004   
3    AAPL 2023-05-22  173.979996  174.710007  173.449997  174.199997   
4    AAPL 2023-05-23  173.130005  173.380005  171.279999  171.559998   
..    ...        ...         ...         ...         ...         ...   
95   MSFT 2023-07-05  335.089996  341.649994  334.730011  338.149994   
96   MSFT 2023-07-06  337.299988  342.989990  335.500000  341.269989   
97   MSFT 2023-07-07  339.320007  341.790009  337.000000  337.220001   
98  

In [5]:
fig = px.line(df, x= 'Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [6]:
fig = px.area(df, x='Date', y='Close',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [7]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=30).mean().reset_index(0, drop=True)

In [9]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [10]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [11]:
# create a DataFrame with the stock prices of Apple and Google
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
google = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'GOOGL'})
df_corr = pd.merge(apple, google, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='GOOGL',
                 trendline='ols',
                 title='Correlation between Apple and Google')
fig.show()